# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [80]:
import collections
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [12]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet101_kitti_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'kitti_label_map.pbtxt')

## Download Model

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [13]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [15]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [41]:
def run_attack(image, graph, lr=0.1):
    with graph.as_default():
        with tf.Session() as sess:
            det_scores = tf.get_default_graph().get_tensor_by_name('detection_scores:0')
            image_tensor = tf.Variable(tf.get_default_graph().get_tensor_by_name('image_tensor:0'))
        
            obj = tf.reduce_mean(det_scores)
            grad = tf.gradients(obj, image_tensor)
            update = -lr * tf.sign(grad)
      
            update_np = sess.run(update, feed_dict={image_tensor: np.expand_dims(image, 0)})
      
    return update_np

In [17]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [42]:
image_path = '/rscratch/data/kitti_2d_object/testing/image_2/001234.png'

image = Image.open(image_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
adv_image = run_attack(image_np, detection_graph)
# Actual detection.
output_dict = run_inference_for_single_image(adv_image, detection_graph)
# Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
  adv_image,
  output_dict['detection_boxes'],
  output_dict['detection_classes'],
  output_dict['detection_scores'],
  category_index,
  instance_masks=output_dict.get('detection_masks'),
  use_normalized_coordinates=True,
  line_thickness=8)
im = Image.fromarray(adv_image)
im.show()

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [None]. Consider casting elements to a supported type.

In [115]:
ge = tf.contrib.graph_editor

with detection_graph.as_default():
    with tf.Session() as sess:
        det_scores = tf.get_default_graph().get_tensor_by_name('detection_scores:0')
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
    #     eps = tf.Variable(tf.zeros((1, 375, 1242, 3)), trainable=True, dtype=tf.float32)
    #     adv_image = tf.add(eps, tf.to_float(image_tensor))

    #     output = image_tensor.op.outputs[0]
    #     ge.detach(ge.sgv([output]))
    #     ge.connect(ge.sgv([adv_image]), ge.sgv([output]))

        obj = tf.reduce_mean(det_scores)
        grad = tf.gradients(obj, image_tensor)
        x = sess.run(obj, feed_dict={image_tensor: np.expand_dims(image_np, 0)})
        y = sess.run(grad, feed_dict={image_tensor: np.expand_dims(image_np, 0)})

InvalidArgumentError: You must feed a value for placeholder tensor 'geph__image_tensor_0_1' with dtype uint8
	 [[node geph__image_tensor_0_1 (defined at /scratch/normanmu/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/contrib/graph_editor/util.py:470)  = Placeholder[dtype=DT_UINT8, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'geph__image_tensor_0_1', defined at:
  File "/scratch/normanmu/miniconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/scratch/normanmu/miniconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/scratch/normanmu/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/scratch/normanmu/miniconda3/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    

In [113]:
grad

[None]

In [106]:
with detection_graph.as_default():
    det_scores = tf.get_default_graph().get_tensor_by_name('detection_scores:0')
    x = tf.gradients(det_scores, det_scores.op.inputs[0])

IndexError: list index out of range

In [116]:
image_np

array([[[ 29,  20,  19],
        [ 28,  25,  14],
        [ 25,  24,  12],
        ...,
        [106,  81,  47],
        [ 91,  74,  56],
        [ 63,  75,  55]],

       [[ 30,  17,  37],
        [ 29,  23,  37],
        [ 29,  22,  37],
        ...,
        [109,  88,  46],
        [ 96,  98,  49],
        [ 80,  87,  47]],

       [[ 16,  17,  38],
        [ 16,  23,  40],
        [ 22,  23,  45],
        ...,
        [ 92,  92,  33],
        [ 81,  90,  35],
        [ 79,  71,  32]],

       ...,

       [[ 79,  82,  94],
        [ 75,  82,  94],
        [ 93,  86,  95],
        ...,
        [ 29,  26,  33],
        [ 29,  23,  37],
        [ 28,  24,  26]],

       [[ 68, 102, 128],
        [ 69, 107, 126],
        [ 99, 116, 123],
        ...,
        [ 29,  23,  22],
        [ 28,  27,  24],
        [ 25,  33,  23]],

       [[167, 150, 124],
        [177, 169, 122],
        [204, 184, 129],
        ...,
        [ 22,  22,  22],
        [ 32,  21,  22],
        [ 43,  31,  22]]